In [ ]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *

In [ ]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

with open('data/reddit-comments-2015-08.csv', 'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    #reader is a list of [[sen123],[sen456],..]
    #eaxh x, ie., x[0] is sen123
    #now sen123 is sent_tokenized to give [sen1,sen2,sen3]
    #*[nltk.sent_tokenize(x[0].lower()) for x in reader], at end you have the below list of sentences. 
    # iterchain will combine the sentences in [[sen1,sen2,sen3],[sen4,sen5,sen6]], this is the input to iterchain
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    #output after iterchain is [sen1,sen2,sen3,sen4,sen5,sen5]
    #print list(sentences)
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]

In [ ]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tokenized_sentences[0], len(tokenized_sentences))

In [ ]:
#import pickle
#with open('data.txt', 'wb') as f:
#    pickle.dump(tokenized_sentences, f)

In [1]:
import pickle
with open('data.txt', 'rb') as fp:
    tokenized_sentences = pickle.load(fp)

In [2]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [3]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [4]:
word = wordlist

In [5]:
embeddings[2761]

array([-0.00781539, -0.96002924,  0.70458364,  0.90016514, -0.25974044,
       -0.82872403,  0.32521391,  1.23866975, -0.2158218 , -0.58259046,
        0.56482399, -0.59560162,  0.07931113, -0.35424623,  0.10623121,
       -0.48939413, -0.4324761 ,  0.6599136 , -0.06735279, -0.42809385,
       -0.11427003,  0.08903392,  0.16611898, -0.15760079,  0.20780087,
        0.18664664,  0.11466923, -0.15877327,  0.14732024,  0.04144473,
       -0.97858113, -0.15506968, -0.39343002,  0.34112266, -0.52571779,
       -1.01909554,  0.63601589, -0.08794332,  0.13931367, -0.10933137,
        0.89080191, -0.11247748, -0.09380047, -0.62214637,  0.22348887,
       -0.45586807, -0.16441527, -0.12084212,  0.75253934,  0.74877995,
       -0.24148768,  0.10676587, -0.34654984, -0.84981006, -0.05637517,
        1.17814636,  0.38551748,  0.75880963,  0.72178942,  0.31493187,
        0.1618239 , -0.6417715 , -0.83980197, -0.14380476, -0.45894837,
        0.07221667, -0.31993574, -0.55760813, -0.599801  ,  0.11

In [6]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [7]:
word_to_index

{'wife': 1080,
 'eats': 7109,
 'laser': 4616,
 'strive': 7418,
 'mentioning': 4027,
 '**|**': 821,
 'reminds': 3201,
 'drafted': 7650,
 'fox': 3317,
 'catfish': 7181,
 'typically': 1370,
 'als': 6338,
 'suggestions': 2129,
 'lands': 4800,
 'draws': 6113,
 'regulation': 6051,
 'reader': 5279,
 'succeed': 5134,
 'surprised': 1206,
 'unlocked': 5151,
 'guarantee': 3245,
 'ps4': 3045,
 'roller': 7548,
 'some': 72,
 'firmly': 5350,
 'grams': 5728,
 'alternative': 2514,
 'claiming': 2450,
 '_____': 7475,
 'gained': 2592,
 'timer': 3367,
 '^^^=': 6825,
 'starters': 4587,
 'wrath': 3067,
 'headaches': 7351,
 'designers': 6544,
 'gio': 7838,
 'gifs': 7704,
 'noting': 7599,
 'disappeared': 7680,
 'hour': 836,
 'conservatives': 4540,
 'tor': 6555,
 'ar': 7651,
 'scripts': 7076,
 'require': 1267,
 'needed': 592,
 'drinks': 3003,
 '*will*': 2887,
 'divine': 5355,
 'clearing': 7789,
 'dragons': 3981,
 'parent': 2610,
 'popularity': 5557,
 'eternity': 7221,
 'rider': 6861,
 'entertaining': 4340,
 'wa

In [8]:
word_to_index['yellow']

2761

In [9]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [10]:
import numpy as np
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [11]:
x_example, y_example = X_train[10], y_train[10]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
SENTENCE_START no one is going to be honest enough to run the check , see they 're a felon , and then all of a sudden immediately turn dishonest and say `` nah , you know what , here 's your gun anyway . ''
[1, 73, 64, 14, 125, 6, 27, 1131, 209, 6, 327, 4, 330, 5, 113, 33, 76, 8, 4912, 5, 9, 85, 53, 10, 8, 3196, 1015, 493, 7835, 9, 134, 50, 3132, 5, 11, 96, 52, 5, 129, 18, 38, 315, 578, 3, 41]

y:
no one is going to be honest enough to run the check , see they 're a felon , and then all of a sudden immediately turn dishonest and say `` nah , you know what , here 's your gun anyway . '' SENTENCE_END
[73, 64, 14, 125, 6, 27, 1131, 209, 6, 327, 4, 330, 5, 113, 33, 76, 8, 4912, 5, 9, 85, 53, 10, 8, 3196, 1015, 493, 7835, 9, 134, 50, 3132, 5, 11, 96, 52, 5, 129, 18, 38, 315, 578, 3, 41, 2]


In [12]:
len(X_train)

79170

In [13]:
X_train

array([ [1, 7, 3574, 8, 156, 795, 26, 224, 9, 33, 21, 204, 5127, 350, 92, 7, 67, 208, 6, 3],
       [1, 12, 18, 8, 3092, 6180, 0, 0, 6180, 3],
       [1, 984, 1476, 228, 598, 16, 774, 3487, 2956, 5, 0, 598, 472, 6061, 5, 491, 598, 472, 6140, 2739, 5, 9, 72, 5702, 16, 0, 0, 3],
       ...,
       [1, 0, 5, 42, 0, 5, 14, 64, 10, 153, 756, 0, 58, 4, 0, 13, 99, 17, 619, 68, 12, 110, 21, 3],
       [1, 39, 145, 3518, 25, 0, 0, 0, 9, 1045, 563, 0, 0, 0, 0, 3],
       [1, 4, 4297, 20, 0, 19, 175, 13, 233, 75, 102, 1290, 15, 25, 162, 9, 13, 7, 161, 17, 132, 4, 563, 69, 12, 18, 785, 6, 27, 0, 3]], dtype=object)

In [14]:
tokenized_sentences[10]

['SENTENCE_START',
 'no',
 'one',
 'is',
 'going',
 'to',
 'be',
 'honest',
 'enough',
 'to',
 'run',
 'the',
 'check',
 ',',
 'see',
 'they',
 "'re",
 'a',
 'felon',
 ',',
 'and',
 'then',
 'all',
 'of',
 'a',
 'sudden',
 'immediately',
 'turn',
 'dishonest',
 'and',
 'say',
 '``',
 'nah',
 ',',
 'you',
 'know',
 'what',
 ',',
 'here',
 "'s",
 'your',
 'gun',
 'anyway',
 '.',
 "''",
 'SENTENCE_END']

In [15]:
import tensorflow as tf
import numpy as np
x = X_train
max_length = max(len(row) for row in x)
x_padded = np.array([row + [0] * (max_length - len(row)) for row in x])
x_tensor = tf.convert_to_tensor(x_padded)

In [16]:
import numpy as np
x = y_train
max_length = max(len(row) for row in x)
y_padded = np.array([row + [0] * (max_length - len(row)) for row in x])
y_tensor = tf.convert_to_tensor(y_padded)

In [17]:
x_padded 

array([[   1,    7, 3574, ...,    0,    0,    0],
       [   1,   12,   18, ...,    0,    0,    0],
       [   1,  984, 1476, ...,    0,    0,    0],
       ..., 
       [   1,    0,    5, ...,    0,    0,    0],
       [   1,   39,  145, ...,    0,    0,    0],
       [   1,    4, 4297, ...,    0,    0,    0]])

In [18]:
y_padded

array([[   7, 3574,    8, ...,    0,    0,    0],
       [  12,   18,    8, ...,    0,    0,    0],
       [ 984, 1476,  228, ...,    0,    0,    0],
       ..., 
       [   0,    5,   42, ...,    0,    0,    0],
       [  39,  145, 3518, ...,    0,    0,    0],
       [   4, 4297,   20, ...,    0,    0,    0]])

In [19]:
import tensorflow as tf
z = [2763,0]
tf.nn.embedding_lookup(embeddings, z)

<tf.Tensor 'embedding_lookup:0' shape=(2, 200) dtype=float32>

In [20]:
tf.InteractiveSession().run(tf.nn.embedding_lookup(embeddings, z))

array([[ 1.12261748,  1.07197416, -0.18724339, -0.09346128, -1.0280689 ,
         0.1638882 , -0.10434432, -0.73139   ,  1.1562748 , -0.30800653,
        -0.69865322, -0.07324108,  0.30612069, -0.25338396, -0.56239402,
        -0.8646791 , -0.44424289,  0.05422308,  0.17322272,  0.94733042,
        -0.10079843, -0.31137192, -0.5167346 , -0.31001687, -1.7446804 ,
         0.93849123,  0.46673664,  0.21381432,  0.07874353,  0.39004678,
         0.97723538, -0.32416728, -0.1724211 ,  1.12379622,  0.75076693,
         0.33819035,  0.00903004,  0.79284203, -1.18947971,  0.12093852,
         0.508798  , -1.32484865,  0.38485217,  0.92830265,  0.07834905,
        -0.98400843,  0.05005199,  0.06951942, -0.41476256, -1.52646422,
        -0.50009418, -0.04572278, -1.37805736, -0.74078822,  0.45445547,
         0.24622086, -1.79271388, -0.74465311, -0.24273418, -0.07206631,
        -0.57831132,  0.66648704, -0.26680511, -0.24319315,  0.9310962 ,
         0.50152546,  0.42568713, -0.77300262,  0.4

In [21]:
len(tf.InteractiveSession().run(tf.nn.embedding_lookup(embeddings, x_padded[:3]))[0][0])

200

In [22]:
len(x_padded),len(y_padded)

(79170, 79170)

In [23]:
xtrain,ytrain = x_padded[:1000], y_padded[:1000]

In [24]:
len(xtrain),len(ytrain)

(1000, 1000)

In [25]:
xtest,ytest = x_padded[1200:1400], y_padded[1200:1400]

In [26]:
len(xtest),len(ytest)

(200, 200)

In [27]:
import tensorflow as tf

In [28]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 
hm_epochs = 150
batch_size = 20
num_steps = 791
state_size = 200
vocab_size = 8000

# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [batch_size])
y = tf.placeholder(tf.int32, [batch_size,None])
keep_prob = tf.constant(1.0)
sequencelength = [len(X_train[i]) for i in X_train][:1000]
testsequencelength = [len(y_train[i]) for i in y_train][1200:1400]
def train_neural_network(x): 
    
        def next_batch(step):
            p,q = xtrain[batch_size*step:batch_size*(step+1)], ytrain[batch_size*step:batch_size*(step+1)]
            r = sequencelength[batch_size*step:batch_size*(step+1)]
            return p,q,r
        
        def test_batch(stp):    
            a,b = xtest[batch_size*stp:batch_size*(stp+1)], ytest[batch_size*stp:batch_size*(stp+1)]
            c = testsequencelength[batch_size*stp:batch_size*(stp+1)]
            return a,b,c
              
        def lstm_neural_network(x):
            # Embedding layer
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            print(rnn_inputs)
            # RNN
            initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                                    initializer=tf.constant_initializer(0.0)),[batch_size,1])
            #inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)   
            cell = tf.contrib.rnn.GRUCell(state_size)
            cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
            print("this is rnn going in:", rnn_inputs)
            print("this is the seqlen:", seqlen)
            rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                                   initial_state=initial_state)

            rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
            print("this is the output:",rnn_output)
            softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
            softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
            logits = tf.matmul(rnn_output, softmax_w) + softmax_b
            return logits

        prediction = lstm_neural_network(x)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [prediction],
            [tf.reshape(y[:batch_size], [-1])],
            [tf.ones([batch_size * num_steps], dtype=tf.float32)])
        cost = tf.reduce_sum(loss) / batch_size
        optimizer = tf.train.AdamOptimizer().minimize(cost)   
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(hm_epochs):
                epoch_loss = 0 
                for step in range(int(len(xtrain)/batch_size)):
                    epoch_x, epoch_y, sl = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y, seqlen: sl})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model/")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(xtest)/batch_size)):
                s,u,v = test_batch(stp)
                correct = tf.equal((tf.argmax(prediction,1)),tf.cast(tf.reshape(tf.concat(axis=1, values= u), [-1]),tf.int64))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                print( 'Batch #', stp, 'Accuracy by each batch:',accuracy.eval({x: s, y: u,seqlen: v}))

In [29]:
train_neural_network(x)

Tensor("embedding_lookup:0", shape=(20, ?, 200), dtype=float32)
this is rnn going in: Tensor("embedding_lookup:0", shape=(20, ?, 200), dtype=float32)
this is the seqlen: Tensor("Placeholder_1:0", shape=(20,), dtype=int32)
this is the output: Tensor("Reshape:0", shape=(?, 200), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 150 loss: 350678.928223
Epoch 1 completed out of 150 loss: 345966.856934
Epoch 2 completed out of 150 loss: 342066.316895
Epoch 3 completed out of 150 loss: 338180.718262
Epoch 4 completed out of 150 loss: 334287.438965
Epoch 5 completed out of 150 loss: 330367.032715
Epoch 6 completed out of 150 loss: 326431.490234
Epoch 7 completed out of 150 loss: 322506.257813
Epoch 8 completed out of 150 loss: 318592.100586
Epoch 9 completed out of 150 loss: 314680.591309
Epoch 10 completed out of 150 loss: 310766.852051
Epoch 11 completed out of 150 loss: 306851.881836
Epoch 12 completed out of 150 loss: 302934.

In [5]:
import tensorflow as tf
saver = tf.train.Saver()        
with tf.Session() as sess:
    saver.restore(sess,"./model")
    print("Model Restored")
    predictin = sess.run(lstm_neural_network, feed_dict={x:x_padded[1001],seqlen: [len(X_train[i]) for i in X_train[1001]]})    

ValueError: No variables to save

### The below code is for static_RNN and with inputs in [num_steps, batch_size, state_size]

In [ ]:
tf.reset_default_graph()

batch_size = 200
num_steps = 791
state_size = 200
vocab_size = 8000


# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [batch_size])
y = tf.placeholder(tf.int32, [batch_size])
keep_prob = tf.constant(1.0)
seqlen = [len(X_train[i]) for i in X_train]


# Embedding layer
rnn_inputs = tf.nn.embedding_lookup(embeddings, x_padded[:batch_size])
print(rnn_inputs)
# RNN
#initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                        #initializer=tf.constant_initializer(0.0)),[batch_size,1])
inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)
print(len(list(inputs)))
cell = tf.contrib.rnn.BasicLSTMCell(state_size) 
rnn_outputs, state = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

#cell = tf.contrib.rnn.GRUCell(state_size)
#rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, sequence_length=seqlen[:batch_size],
                                                # initial_state=initial_state, time_major=True)
#print("this is output:", rnn_outputs)
print("the lenght of output:", len(rnn_outputs))

rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
print("this is the output:",rnn_output)
softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
logits = tf.matmul(rnn_output, softmax_w) + softmax_b

### The below code is using a dynamic RNN with inputs [batch_size, num_steps, state_size] , this has some issue when the input is in [num_steps,batch_size,state_size] as we have to put time_major = True when we use this. 

Source 1: http://stackoverflow.com/questions/42513613/tensorflow-dynamic-rnn-regressor-valueerror-dimension-mismatch
Source 2: https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn

In [ ]:
tf.reset_default_graph()

batch_size = 200
num_steps = 791
state_size = 200
vocab_size = 8000


# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [batch_size])
y = tf.placeholder(tf.int32, [batch_size])
keep_prob = tf.constant(1.0)
seqlen = [len(X_train[i]) for i in X_train]


# Embedding layer
rnn_inputs = tf.nn.embedding_lookup(embeddings, x_padded[:batch_size])
print(rnn_inputs)
# RNN
initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                        initializer=tf.constant_initializer(0.0)),[batch_size,1])
#inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)
#print(len(list(inputs)))
#cell = tf.contrib.rnn.BasicLSTMCell(state_size) 
#rnn_outputs, state = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

cell = tf.contrib.rnn.GRUCell(state_size)

rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen[:batch_size],
                                                 initial_state=initial_state)
#print("this is output:", rnn_outputs)
print("the lenght of output:", rnn_outputs)

rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
print("this is the output:",rnn_output)
softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
logits = tf.matmul(rnn_output, softmax_w) + softmax_b
logits

In [ ]:
logits
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(y_padded[:batch_size], [-1])],
        [tf.ones([batch_size * num_steps], dtype=tf.float32)])
cost = tf.reduce_sum(loss) / batch_size
optimizer = tf.train.AdamOptimizer().minimize(cost)

### The below code is for dynamic_rnn with inputs as time_major = True

In [ ]:
tf.reset_default_graph()

batch_size = 200
num_steps = 791
state_size = 200
vocab_size = 8000


# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [batch_size])
y = tf.placeholder(tf.int32, [batch_size])
keep_prob = tf.constant(1.0)
seqlen = [len(X_train[i]) for i in X_train]


# Embedding layer
rnn_inputs = tf.nn.embedding_lookup(embeddings, x_padded[:batch_size])
print(rnn_inputs)
# RNN
initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                        initializer=tf.constant_initializer(0.0)),[batch_size,1])
inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)
#print(len(list(inputs)))
#cell = tf.contrib.rnn.BasicLSTMCell(state_size) 
#rnn_outputs, state = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

cell = tf.contrib.rnn.GRUCell(state_size)

rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, sequence_length=seqlen[:batch_size],
                                                 initial_state=initial_state, time_major = True)
#print("this is output:", rnn_outputs)
print("the lenght of output:", rnn_outputs)

rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
print("this is the output:",rnn_output)
softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
logits = tf.matmul(rnn_output, softmax_w) + softmax_b